My pseudocode for getting betting data for one season - 

For each team:

    For each game:
    
        1. Get date and opposing team
        
        2. Get list of games already in CSV which occurred on that date
        
        3. If no such games exists,
        
            add this game to the list
        
        4. Else
            
            a. search if this particular game exists
            
            If no, add this game to list
            
            If yes, 
            
                if current team is home team, populate homeLine column.
                if current team is away team, populate awayLine column

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os
import shutil
import unicodedata
import mlbgame
import glob

In [285]:
year = 1999
filename = 'Betting Data/'+str(year)+'.csv'

In [286]:
#these are codes used to access the covers.com website
teams = {
    #AL Central
    'CLE': 2980,
    'CHW': 2974,
    'DET': 2978,
    'KC': 2965,
    'MIN': 2983,
    
    #AL West
    'HOU': 2981,
    'TEX': 2977,
    'LAA': 2979,
    'OAK': 2969,
    'SEA': 2973,
    
    #AL East
    'BAL': 2959,
    'BOS': 2966,
    'NYY': 2970,
    'TB': 2960,
    'TOR': 2984,
    
    #NL Central
    'CHC': 2982,
    'CIN': 2961,
    'MIL': 2976,
    'PIT': 2971,
    'STL': 2975,
    
    #NL West
    'ARI': 2968,
    'COL': 2956,
    'LAD': 2967,
    'SD': 2955,
    'SF': 2962,
    
    #NL East
    'ATL': 2957,
    'MIA': 2963,
    'NYM': 2964,
    'PHI': 2958,
    'WAS': 2972
}

In [287]:
rows_list = []
rows_list.append({
            'date':'11/11/1111',
            'homeTeam': 'Hi',
            'awayTeam': 'Hi',
            'homeScore':0,
            'awayScore': 0,
            'homePitcher':'Hi',
            'awayPitcher':'Hi',
            'homeLine': 0,
            'awayLine': 0
        })
print (rows_list)
f = pd.DataFrame(rows_list)
f.to_csv(filename,  index = None, columns = ['date', 'homeTeam', 'awayTeam', 'homeScore', 'awayScore', 'homePitcher',
                                              'awayPitcher', 'homeLine', 'awayLine'])

[{'awayLine': 0, 'awayScore': 0, 'awayPitcher': 'Hi', 'date': '11/11/1111', 'homeScore': 0, 'homePitcher': 'Hi', 'awayTeam': 'Hi', 'homeTeam': 'Hi', 'homeLine': 0}]


In [288]:
def get_current_home_away_teams(team, opposing_team):
    home = ""
    away = ""
    if opposing_team[:2] == "@ ":
        home = opposing_team[2:]
        away = team
    else:
        home = team
        away = opposing_team
        
    if (home=="LA"):
        home = "LAD"
    if (away=="LA"):
        away="LAD"
        
    return home, away

In [289]:
def get_scores(scores, isCurrentTeamHome):
    scores = unicodedata.normalize("NFKD", scores.strip())
    scores = scores.split(" ")[1].split("-")
    if (isCurrentTeamHome):
        home_score = scores[0]
        away_score = scores[1]
    else:
        home_score = scores[1]
        away_score = scores[0]
    return home_score, away_score

In [290]:
def get_moneylines(moneyline, isCurrentTeamHome):
    if (isCurrentTeamHome):
        homeline = moneyline
        awayline = 0
    else:
        homeline = 0
        awayline = moneyline
    return homeline, awayline

In [291]:
def scrape_info_for_team(team):
    
    #first reading existing csv
    df = pd.read_csv(filename)
    
    #get webpage for current team
    query = 'https://www.covers.com/pageLoader/pageLoader.aspx?page=/data/mlb/teams/pastresults/'+str(year)+'/team' + str(teams[team])+'.html'
    r = requests.get(query)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #getting all games played by team
    games = soup.find_all("tr", class_=lambda x: x !='datahead')
    games_to_be_added = []
    
    for game in games:
        current_date = ""
        current_home_team = ""
        current_away_team=""
        current_home_score=0
        current_away_score=0
        current_home_pitcher=""
        current_away_pitcher=""
        current_home_line = 0
        current_away_line = 0
        
        
        cells = game.find_all(class_='datacell')
        
        #get date (first check for existence of game in database)
        current_date = unicodedata.normalize("NFKD", (cells[0].text).strip())
        
        #get teams (second check)
        opposing_team = (cells[1].text).strip()
        current_home_team, current_away_team = get_current_home_away_teams(team, opposing_team)
        
        #seeing if the game was a tie
        if (current_date=="09/29/16" and current_home_team=="PIT" and current_away_team=="CHC"):
            continue
        if (current_date=="06/30/05" and current_home_team=="CIN" and current_away_team=="HOU"):
            continue
        if (current_date=="09/30/01" and current_home_team=="NYY" and current_away_team=="BAL"):
            continue
        
        #get scores
        scores = (cells[2].text)
        #print (current_date, scores)
        current_home_score, current_away_score = get_scores(scores, (current_home_team==team))
        
        #get pitchers
        if (cells[3].a):   
            current_away_pitcher = (cells[3].a.text).strip()
        else:
            current_away_pitcher = (cells[3].text).strip()[:-4]
        if (cells[4].a):
            current_home_pitcher = (cells[4].a.text).strip()
        else:
            current_home_pitcher = (cells[4].text).strip()[:-4]
        
        #get line for current team and current game
        current_home_line, current_away_line = get_moneylines((cells[5].text).strip()[2:], (current_home_team==team))
        
        
        
        #see if there are any games in existing CSV which were played on this date
        games_on_this_date = df[df['date']==current_date]
            
        #check if this exact game is in database
        filter1 = games_on_this_date[games_on_this_date['homeTeam']==current_home_team] #same home team
        filter2 = filter1[filter1['homePitcher']==current_home_pitcher] #same home pitcher
        filter3 = filter2[filter2['date']==current_date] #same date
        if (filter3.shape[0]==0): #this exact game is NOT already in database
                    games_to_be_added.append({
                        'date':current_date,
                        'homeTeam': current_home_team,
                        'awayTeam': current_away_team,
                        'homeScore': current_home_score,
                        'awayScore': current_away_score,
                        'homePitcher': current_home_pitcher,
                        'awayPitcher': current_away_pitcher,
                        'homeLine': current_home_line,
                        'awayLine': current_away_line
                })
        else: #this exact game IS in database. Need to update moneyline
            if (current_away_team == team):
                new_df = {
                    'date': filter3['date'],
                    'homeTeam': filter3['homeTeam'],
                    'awayTeam': filter3['awayTeam'],
                    'homeScore': filter3['homeScore'],
                    'awayScore':filter3['awayScore'],
                    'homePitcher': filter3['homePitcher'],
                    'awayPitcher': filter3['awayPitcher'],
                    'homeLine': filter3['homeLine'],
                    'awayLine': current_away_line
                }
                new_df = pd.DataFrame(new_df)
                df.update(new_df)
            else:
                new_df = {
                    'date': filter3['date'],
                    'homeTeam': filter3['homeTeam'],
                    'awayTeam': filter3['awayTeam'],
                    'homeScore': filter3['homeScore'],
                    'awayScore':filter3['awayScore'],
                    'homePitcher': filter3['homePitcher'],
                    'awayPitcher': filter3['awayPitcher'],
                    'homeLine': current_home_line,
                    'awayLine': filter3['awayLine']
                }
                new_df = pd.DataFrame(new_df)
                df.update(new_df)
                
    df2 = pd.DataFrame(games_to_be_added)
    df = df.append(df2, ignore_index=True)
    return df
        

In [292]:
for team, initials in teams.items():
    print (team)
    x = scrape_info_for_team(team)
    x.to_csv(filename, index=None, columns = ['date', 'homeTeam', 'awayTeam', 'homeScore', 'awayScore', 'homePitcher',
                                           'awayPitcher', 'homeLine', 'awayLine'])

TOR
HOU
ATL
BAL
OAK
BOS
ARI
LAA
CLE
STL
SD
COL
DET
TEX
MIN
CIN
SF
TB
KC
LAD
SEA
PIT
CHW
WAS
NYM
PHI
MIL
CHC
NYY
MIA


# Removing the filler rows in each betting CSV

In [12]:
years = [i for i in range(1999, 2018)]

for year in years:
    current_year = pd.read_csv('Betting Data/'+ str(year) + '.csv')
    current_year = current_year[current_year.homeTeam != 'Hi']
    current_year.to_csv('Betting Data/'+ str(year) + '.csv', index = None)